In [2]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt

# Define parameters 

In [24]:
# initial counts of epithelial cells 
Es0 = 250*10e6 # tracheal epithelial cells 
Ei0 = 0 # begin with no infected cells 
Ep0 = 0 # no phagocytosed cells 

# initial inoculation of virus 
v0 = 10e4 # range from 10**3 to 10**7 RNA copies/mL or max of approx 10**4 M

# rate of viral dissemination 
delta = 1 # starting  with 1, need to do some digging to see what a better number is given rapid 
          # spread of human parainfluenza 

# contact matrix between virion and susceptible respiratory epithelial cells
c_vEs = np.array([
    [1, 0],
    [0, 1]
])

# probability that a virion in contact with a susceptible cell will gain entry 
p_entry = 0.8 # must vary between 0 and 1... maybe pull from a probability distribution? 

# rate of viral budding
b = 1 # placeholder

# viral replication rate
r = 1 # placeholder

# probability of phagocytosis
p_phagocytosis = 0.5 # maybe pull from a probability distribution? 

# rate of respiratory epithelial cell synthesis 
n_epithelial = 1 # placeholder 

# rate of epithelial cell infection
beta = 2

# rate of recovery of infected cells
gamma = 0.5

# rate of phagocytosis 
rho = 1

# contact matrix between infected cells and activated T cells 
c_EiT = np.array([
    [1, 0],
    [0, 1]
])

# rate of M_0 synthesis 
# n_macrophage = 

# rate of differentiation from M_0 to M_1
# nu_01 = 

# rate of differentiation from M_0 to M_2 
# nu_02 =

# rate of differentiation from M1 to M2
# nu_12 = 

# rate of differentiation fomr M2 to M1
# nu_21 = 

# rate of T cell synthesis
# n_T = 

# probability of T cell activation 
# p_activation = # function of p_endocytosis  

# rate of T cell exhaustion 
# e =

# rate of T cell death 
# u_T = 

# rate of pro inflammmatory cytokine production
# mu_cpro = 

# rate of cpro endocytosis by T cell 
# zeta_pro = 

# rate of canti endocytosis by T cell 
# zeta_anti = 

# contact matrix between cpro and activate T cell
# c_cproT =

# contact matrix between canti and activated T cell
# c_cantiT = 

# probability of cytokine endocytosis by a T cell
# p_endocytosis

# rate of M1/M2 expression/endocytosis of TREM2
# mu_M1exp = 
# mu_M1endo = 

# mu_M2exp = 
# mu_M2endo = 

# cleavage rate
# eta = 

# contact matrix between M1 TREM2 and a cleaving enzyme/cytokine
# c_M1enzyme =

# rate of apoptosis for macrophages
# a0 = 

# a1 =
 
# a2 = 

# probability of apoptosis for macrophages
# p_apoptosisM0 = 

# p_apoptosisM1 =

# p_apoptosisM2 = 

# TREM2 receptors / cell 
rec = 10e4 # what chatGPT said when I told it to "speculate"

# probability of cleavage
#p_cleavage = 

# rate of sTREM2 degradation
# mu_sTREM2 = 

# Define ODEs

## Epithelial cells - use Forward Euler SIR with birth and death 

In [15]:
def epithelial(Es0, Ei0, Ep0, beta, gamma, t_final, dt):
    
    T = np.linspace(0, t_final, 500)
    Es = np.zeros(len(T))
    Ei = np.zeros(len(T))
    Ep = np.zeros(len(T))
    
    Es[0] = Es0
    Ei[0] = Ei0
    Ep[0] = Ep0 
    
    N = Es0 + Ei0 + Ep0 # Ep stands for phagocytosed, analogous to recovered
    
    for i in range(len(T)):
        if i == 0:
            Es[i] = Es0
            Ei[i] = Ei0
            Ep[i] = Ep0
        
        else: 
            Es[i] = Es[i-1] + dt*(n_epithelial - beta*Es[i-1]*Ei[i-1] / N) 
            Ei[i] = Ei[i-1] + dt*((beta*Es[i-1]*Ei[i-1] / N) - gamma*Ei[i-1])
            Ep[i] = Ep[i-1] + dt*(gamma*Ei[i-1])
            
    return Es, Ei, Ep, T

## ODE to track virions

In [8]:
def virus(v0, ve, Ei, t):
    
    # soluble virus
    dvsdt = v0 - delta*cvEs*p_entry + b*Ei
    
    # endocytosed virus
    dvedt = delta*cvEs*p_entry + r*ve - b*Ei - rho*cEiT*p_phagocytosis 

    return [dvsdt, dvedt]

## ODEs to track macrophages

In [ ]:
def macrophage(M0,M1,M2 t):
    #M0
    dM0dt = -M0*(nu_01 + nu_02 -a0*p_apoptosisM0) + n_macrophage*M0 
    # question: should this last term be n_macrophage*(M0 +M1+M2)? Can only M0
    # macrophages synthesize new M0 macrophages or can any? 
    
    #M1
    dM1dt = nu_01*M0 - M1*(nu_12 + a1*p_apoptosisM1)
    
    #M2
    dM2dt = nu_02*M0 - M2*(nu_21 + a2*p_apoptosisM2)
    
    return [dM0dt, dM1dt, dM2dt]

## ODEs to track T cells

In [ ]:
def Tcell(Tu, Ta):
    #Tu 
    dTudt = n_T - Tu*p_activation + e*Ta - u_T*(Tu)
    
    #Ta 
    dTadt = Tu*p-activation - e*Ta - u_T*(Ta)
    
    return [dTudt, dTadt]
    

## ODEs to track cytokines 

In [ ]:
def cytokines(M1, M2):
    # pro inflammatory 
    dcprodt = mu_cpro*M1 - zeta_pro*c_cproT*p_endocytosis
    
    # anti inflammatory 
    dcantidt = mu_canti*M2 - zeta_anti*c_cantiT*p_endocytosis 
    
    return [dcprodt, dcantidt]

## ODEs to track TREM2/sTREM2

In [ ]:
def TREM(M1, M2, sTREM2):
    # TREM2
    dTREM2dt = rec*M1*(mu_M1exp - mu_M1endo) + rec*M2*(mu_M2exp - mu_M2endo) - eta(c_M1enzyme + c_M2enzyme)*p_cleavage
    
    # sTREM2
    dsTREM2dt = eta*(c_M1enzyme + c_M2enzyme)*p_cleavage - mu_sTREM2*sTREM2
    
    return [dTREM2dt, dsTREM2dt]

## First attempt: track epithelials

In [21]:
max_time = 50 
dt = 0.1

test_output = epithelial(Es0, Ei0, Ep0, beta, gamma, max_time, dt)

Es = test_output[0]
Ei = test_output[1]
Ep = test_output[2]
T = test_output[3]

## Track virions 

In [25]:
# example syntax for odeint:
# sol = odeint(my_ode, y0, t, args=(arg1, arg2))
t = np.linspace(0, 3600, 100)

output_virions = odeint(virus, 0, t, args=(v0, ve, Ei, t ))
# need to define ve somewhere 

vs = output_virions[:,0]
ve = output_virions[:,1]


NameError: name 've' is not defined

## Eventually: plot something 

In [ ]:
plt.figure(figsize=(8, 6))  

plt.plot(t, vs, label='soluble virus(t)')
plt.plot(t, vt, label='endocytosed virus(t)')

plt.xlabel('Time (t)')
plt.ylabel('vs(t) or ve(t)')
plt.legend()

plt.grid(True)
plt.show()